In [ ]:
# | default_exp climate_utils

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import is_close

In [ ]:
# | export
import warnings
import operator
import collections
import numpy as np
from math import pi
from typing import List, Dict
from numpy import exp, cos, sin, arccos, arctan

In [ ]:
# | export
def compute_vpd_from_t_rh(
    relative_humidity: float,  # Air relative_humidity (%)
    temperature: float,  # Air temperature (degrees Celsius)
    air_pressure: float = 101325,  # Unknown parameter definition Air pressure, used?
) -> float:
    "Compute vapor pressure deficit (VPD) from air relative humidity and air temperature"

    # Assert parameters ---------------------------------------------------------

    # relative_humidity

    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(relative_humidity),
        0,
        err_msg="\nrelative_humidity must be must be a integer value between 0-100\n",
    )

    np.testing.assert_array_less(
        np.array(relative_humidity),
        101,
        err_msg="\nrelative_humidity must be must be a integer value between 0-100\n",
    )

    # temperature
    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(temperature),
        -100,
        err_msg="\ntemperature must be must be value between -100 and 100\n",
    )

    np.testing.assert_array_less(
        np.array(temperature),
        101,
        err_msg="\ntemperature must be must be value between -100 and 100\n",
    )

    # Define Constants ----------------------------------------------------------

    # molar weight dry air (g/mol)
    mass = 28.966

    # molar weight H20 H2O(g/mol) Mh2o not used in this function??
    # mass_h2o = 18

    # Perfect gas constant %J/mol/K
    rgz = 8.314472

    # conversion of temperature in K
    temp_kelvin = temperature + 273.15

    # D_air not used in this function??
    # d_air = ((air_pressure) / (rgz * (temp_kelvin))) * mass

    es = 6.108 * exp(17.27 * temperature / (237.2 + temperature)) * 100

    ea = relative_humidity * es / 100

    # Compute VPD ---------------------------------------------------------------

    vpd = (es - ea) / 1000

    # Step implemented in case vpd is a single value i.e. vpd = 41
    if isinstance(vpd, float):
        # Transform float into array
        vpd = np.array([vpd])

    # Remove negative values if exits
    vpd[vpd < 0] = 0

    return vpd

#### __Example: Compute VPD__

In [ ]:
compute_vpd_from_t_rh(relative_humidity=80, temperature=25)

array([0.6339534])

In [ ]:
compute_vpd_from_t_rh(relative_humidity=25, temperature=np.array([80, 90]))

array([35.69261567, 52.96534572])

In [ ]:
# | hide

# Test that function return a positive value
assert compute_vpd_from_t_rh(relative_humidity=80, temperature=25) > 0

In [ ]:
# | hide

# Test that function return a known anwser
is_close(
    compute_vpd_from_t_rh(relative_humidity=80, temperature=25),
    # Value from https://cales.arizona.edu/vpdcalc/
    0.634,
    eps=1e-02,
)

True

In [ ]:
# | export


def compute_pet(
    tmoy: float,  # Mean temperature over the considered time step (degrees Celsius)
    net_radiation: float,  # Cumulative Net radiation over the considered  time sep (MJ.m2)
    g: float = 0,  # Unknown parameter definition
    vpd: float = None,  # Vapor pressure deficit (kpa) for calculating etp using the Penmman (pm) formulation
    wind_speed_u: float = None,  #  Wind speed (m.s-1) for calculating etp using the Penmman (pm) formulation
    pt_coeff: float = None,  # An empirical constant accounting for the vapor pressure deficit and resistance values. Typically, α is 1.26 for open bodies of water, but has a wide range of values from less than 1 (humid conditions) to almost 2 (arid conditions).
    formulation: str = [ # String indicating which formulation to use (Pristeley Taylor (pt) or Penmman (pm)) for calculating etp
        "pt",
        "pm",
    ],
) -> float:  #  Potential evapotranspiration (PET) (mm)
    "Calcule Potential Evapotranspiration (mm) PET using Pristeley Taylor (pt) or Penmman (pm) Formulation"

    # Assert parameters ---------------------------------------------------------
    # tmoy
    assert isinstance(tmoy, float) | isinstance(tmoy, int) | isinstance(tmoy, np.ndarray), "tmoy parameter must be a float or integer value"

    # net_radiation
    assert isinstance(net_radiation, float) | isinstance(net_radiation, int) | isinstance(net_radiation, np.ndarray), "net_radiation parameter must be a float or integer value"

    # g
    assert isinstance(g, float) | isinstance(g, int) | isinstance(g, np.ndarray), "g parameter must be a float or integer value"

    # Formulation
    assert (
        formulation in ["pt", "pm"]
    ), f'{formulation} not a valid option for formulation, select "pt" for Pristeley Taylor (pt) or "pm" for Penmman'

    # wind_speed
    if formulation == "pm":
        assert (
            isinstance(wind_speed_u, float) | isinstance(wind_speed_u, int) | isinstance(wind_speed_u, np.ndarray)
        ), "Parameter wind_speed_u required for pm formulation. This must be a float or integer value"

        # vpd
        assert (
            isinstance(vpd, float) | isinstance(vpd, int) | isinstance(vpd, np.ndarray)
        ), "Parameter vpd required for pm formulation. This must be a float or integer value"

    # pt_coeff
    if formulation == "pt" and pt_coeff is None:
        warnings.warn("Using default value of 1.26 for pt_coeff")
        pt_coeff = 1.26

    elif formulation == "pt" and pt_coeff is not None:
        assert (
            isinstance(pt_coeff, float) | isinstance(pt_coeff, int) | isinstance(pt_coeff, np.ndarray)
        ), "Parameter pt_coeff required for pt formulation. This must be a float or integer value"

    # Calculate pet -------------------------------------------------------------

    # Pristeley-Taylor formulation
    if formulation == "pt":
        # Define Constants

        # Stefan-Boltzman constant [MJ.K^-4.m^-2.day^-1]
        # sb_constant = 4.903 * 10**9

        # Psychometer constant
        gamma = 0.0666

        # Latent heat of vaporisation
        lamb = 2.45

        #  s: slope of the saturation vapour pressure function (AO 1998)
        slope_sta = (
            4098
            * 0.6108
            * exp((17.27 * tmoy) / (tmoy + 237.3))
            / ((tmoy + 237.3) ** 2)
        )

        # Return ETP
        return (
            pt_coeff
            * (slope_sta / (slope_sta + gamma))
            * ((net_radiation - g) / lamb)
        )

    # Penman formulation
    elif formulation == "pm":
        # Define Constants

        # Stefan-Boltzman constant [MJ.K^-4.m^-2.day^-1]
        # sb_constant = 4.903 * 10**9

        # Psychometer constant
        gamma = 0.0666

        # Latent heat of vaporisation
        # lambda = 2.45

        #  s: slope of the saturation vapour pressure function (AO 1998)
        delta = (
            4098
            * 0.6108
            * np.exp((17.27 * tmoy) / (tmoy + 237.3))
            / ((tmoy + 237.3) ** 2)
        )

        # ga = 0.34 * max(u, 0.001)

        u2 = wind_speed_u * (4.87 / np.log(67.8 * 10 - 5.42))

        n1 = 0.408 * delta * net_radiation
        n2 = gamma * (37 / (tmoy + 273)) * u2 * vpd
        d = delta + gamma * (1 + 0.34 * u2)

        # Return pet
        return (n1 + n2) / (d)

#### __Example: Compute Potential Evapotranspiration__

In [ ]:
compute_pet(tmoy=80, net_radiation=25, formulation="pt")

NameError: name 'np' is not defined

In [ ]:
# | hide

# Test that function return a positive value
assert compute_pet(tmoy=80, net_radiation=25, formulation="pt") > 0

/tmp/ipykernel_8746/2528486195.py:52: UserWarning: Using default value of 1.26 for pt_coeff
  warnings.warn("Using default value of 1.26 for pt_coeff")


In [ ]:
# | hide

# Test that function return a known anwser
is_close(
    compute_pet(tmoy=80, net_radiation=25, formulation="pt"), 12.4292, eps=1e-02
)

/tmp/ipykernel_8746/2528486195.py:52: UserWarning: Using default value of 1.26 for pt_coeff
  warnings.warn("Using default value of 1.26 for pt_coeff")


True

In [ ]:
compute_pet(tmoy=20, net_radiation=1, wind_speed_u=2, vpd=2, formulation="pm")

0.3434387636191171

In [ ]:
# | hide
# Test that function return a positive value
assert (
    compute_pet(
        tmoy=20, net_radiation=1, wind_speed_u=2, vpd=2, formulation="pm"
    )
    > 0
)

In [ ]:
# | hide

# Test that function return a known anwser
is_close(
    compute_pet(
        tmoy=20, net_radiation=1, wind_speed_u=2, vpd=2, formulation="pm"
    ),
    0.3434,
    eps=1e-02,
)

True

In [ ]:
# | export
def day_length(
    latitude: float,  # numeric value specifying the geographic latitude (in decimal degrees) of the location of interest
    day_of_year: int,  # numeric (usually integer) value or vector specifying the Julian day (day of the year), for which calculations should be done.
    no_times_as_na: bool = False,  # parameter to determine whether for days without sunrise or sunset, na should be returned for Sunset and Sunrise. If left at FALSE, the function returns -99 and 99 for sunrise and sunset or polar nights and polar days, respectively
) -> Dict:  # Dictionary with three elements Sunrise, Sunset and Daylength. For days without sunrise (polar nights),sunset and sunrise become -99 and the daylength 0. For days without sunset, sunset and sunrise are 99 and daylength 24.
    "Original function from chillR R package. This function computes sunrise time, sunset time and daylength for a particular location and day of the year (Julian day). This is done using equations by Spencer (1971) and Almorox et al. (2005)."

    warnings.warn("Double check day_length function works for Australia")

    # Assert parameters ---------------------------------------------------------
    # Latitude
    assert (
        isinstance(latitude, float) | isinstance(latitude, int)
        and 95 >= latitude >= -95
    ), "Provide latitude as coordinates points bewteen -90 and 90 i.e. latitude = 41.40338"

    # Day of year
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(day_of_year),
        0,
        err_msg="\nday_of_year must be must be a integer value between 1-366\n",
    )

    np.testing.assert_array_less(
        np.array(day_of_year),
        367,
        err_msg="\nError: day_of_year must be must be a integer value between 1-366\n",
    )

    # Define constants ----------------------------------------------------------
    gamma = 2 * (pi / 365) * ((day_of_year) - 1)
    delta = (180 / pi) * (
        0.006918
        - 0.399912 * cos(gamma)
        + 0.070257 * sin(gamma)
        - 0.006758 * cos(2 * gamma)
        + 0.000907 * sin(2 * (gamma))
        - 0.002697 * cos(3 * (gamma))
        + 0.00148 * sin(3 * (gamma))
    )

    cos_wo = (
        sin((-0.8333 / 360) * 2 * pi)
        - sin(latitude / 360 * 2 * pi) * sin((delta / 360) * 2 * pi)
    ) / (cos((latitude / 360) * 2 * pi) * cos((delta / 360) * 2 * pi))

    # Step implemented in case day_of_year is a single value i.e.
    # day_of_year = 80
    if isinstance(cos_wo, float):
        # Transform float into array
        cos_wo = np.array([cos_wo])

    # Calculate sunrise and sunset ----------------------------------------------

    # Get the position of values between -1 and 1
    # Note only day_of_year values from 77 to 80 and will return values
    # between -1 and 1

    normal_days = np.where((cos_wo >= -1) & (cos_wo <= 1))

    # Initialize a numpy arrays

    sunrise = np.full(len(cos_wo), -99, dtype=float)
    sunset = np.full(len(cos_wo), -99, dtype=float)

    # Add normal days to sunrise and sunset arrays
    sunrise[normal_days] = 12 - arccos(cos_wo[normal_days]) / (15 / 360 * 2 * pi)
    sunset[normal_days] = 12 + arccos(cos_wo[normal_days]) / (15 / 360 * 2 * pi)

    # Calculate day length
    day_length = sunset - sunrise

    day_length[np.where(cos_wo > 1)] = 0
    day_length[np.where(cos_wo < (-1))] = 24

    # Replace values with 99
    sunrise[np.where(day_length == 24)] = 99
    sunset[np.where(day_length == 24)] = 99

    # Replace 99 and -99 with na in sunrise and sunset arrays
    if no_times_as_na is True:
        sunrise = np.where((sunrise == -99) | (sunrise == 99), np.nan, sunrise)
        sunset = np.where((sunset == -99) | (sunset == 99), np.nan, sunset)

    # # Check the presence of nan
    if (
        any(np.isnan(sunrise))
        or any(np.isnan(sunset))
        or any(np.isnan(day_length))
    ):
        warnings.warn("nan found in sunrise, sunset or day_length")

    # Return dictionary
    return collections.defaultdict(
        list, {"sunrise": sunrise, "sunset": sunset, "day_length": day_length}
    )

#### __Example: Calculate day length__

This function was originally developed in the ChillR R [package](https://cran.r-project.org/web/packages/chillR/chillR.pdf)

In [ ]:
day_length(latitude=45, day_of_year=100, no_times_as_na=False)

/tmp/ipykernel_8746/1596622700.py:9: UserWarning: Double check day_length function works for Australia
  warnings.warn("Double check day_length function works for Australia")


defaultdict(list,
            {'sunrise': array([5.4049135]),
             'sunset': array([18.5950865]),
             'day_length': array([13.190173])})

In [ ]:
day_length(
    latitude=50,
    day_of_year=np.array([78, 79, 80, 81, 82]),
    no_times_as_na=False,
)

/tmp/ipykernel_8746/1596622700.py:9: UserWarning: Double check day_length function works for Australia
  warnings.warn("Double check day_length function works for Australia")


defaultdict(list,
            {'sunrise': array([5.98160544, 5.95020317, 5.91880891, 5.88742546, 5.85605558]),
             'sunset': array([18.01839456, 18.04979683, 18.08119109, 18.11257454, 18.14394442]),
             'day_length': array([12.03678911, 12.09959365, 12.16238217, 12.22514908, 12.28788884])})

In [ ]:
# | export


def calculate_radiation_diurnal_pattern(
    time_of_day: int,  # Numeric value of vector indicating the time of the day (in seconds)
    day_length: int,  # Value indicating the duration of the day (in seconds). Calculated using the `day_length` function
) -> float:
    "Calculated diurnal pattern of temperature assuming a sinusoidal pattern with T = tmin at sunrise and T = (tmin + tmax)/2 at sunset. From sunset to sunrise follows a linear trend"

    # Assert parameters ---------------------------------------------------------

    # time_of_day

    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as


    if time_of_day < 0:
        warnings.warn('time_of_day is a negative value. Not sure if this is correct')
    #np.testing.assert_array_compare(
    #    operator.__ge__,
    #    np.array(time_of_day),
    #    0,
    #    err_msg="\ntime_of_day must be must be value between 0 and 86400\n",
    #)
    #
    #np.testing.assert_array_less(
    #    np.array(time_of_day),
    #    86401,
    #    err_msg="\ntime_of_day must be must be value between 0 and 86400\n",
    #)

    # day_length
    np.testing.assert_array_compare(
        operator.__ge__,
        np.array(day_length),
        0,
        err_msg="\nday_length must be must be value between 0 and 86400\n",
    )

    np.testing.assert_array_less(
        np.array(day_length),
        86401,
        err_msg="\nday_length must be must be value between 0 and 86400\n",
    )

    # Raise warning for time_of_day and day_length
    warnings.warn("Double check that time_of_day and day_length are in seconds")

    # Calculate_radiation_diurnal_pattern ---------------------------------------

    # sunrise
    ws = (day_length / 3600.0) * (pi / 24.0)
    w = ws - (time_of_day / day_length) * (ws * 2.0)

    prop = ((pi / 24.0) * (cos(w) - cos(ws))) / (sin(ws) - ws * cos(ws))

    return prop / 3600.0

#### __Example: Calculate radiation diurnal pattern__

In [ ]:
calculate_radiation_diurnal_pattern(time_of_day=1, day_length=40)

/tmp/ipykernel_8746/2676460645.py:50: UserWarning: Double check that time_of_day and day_length are in seconds
  warnings.warn("Double check that time_of_day and day_length are in seconds")


0.0036562495459173377

In [ ]:
# | export


def calculate_temperature_diurnal_pattern(
    time_of_day: float,  # Value of vector indicating the time of the day (in seconds from sunrise)
    day_length: int,  # Value indicating the duration of the day (in seconds). Calculated using the `day_length` function
    tmin: float,  # Minimum temperature (in degrees C) of the target day of the year
    tmax: float,  # Maximum temperature (in degrees C) of the target day of the year
    tmin_prev: float,  # Minimum temperature (in degrees C) of the previous target day of the year
    tmax_prev: float,  # Maximum temperature (in degrees C) of the previous target day of the year
    tmin_next: float,  # Minimum temperature (in degrees C) of the next target day of the year
) -> float:
    "Calculated diurnal pattern of temperature assuming a sinusoidal pattern with T = tmin at sunrise and T = (tmin+tmax)/2 at sunset. From sunset to sunrise follows a linear trend"

    # Raise warning for time_of_day and day_length
    warnings.warn("Double check that time_of_day and day_length are in seconds")

    # calculate_temperature_diurnal_pattern -------------------------------------

    if time_of_day < 0.0 or time_of_day >= day_length:
        tfin = 86400.0 - day_length

        if time_of_day < 0.0:
            time_of_day = time_of_day + 86400.0 - day_length

            # Return Temp
            return 0.5 * (tmax_prev + tmin_prev) * (
                1.0 - (time_of_day / tfin)
            ) + tmin * (time_of_day / tfin)

        else:
            time_of_day = time_of_day - day_length

            # Return Temp
            return 0.5 * (tmax + tmin) * (
                1.0 - (time_of_day / tfin)
            ) + tmin_next * (time_of_day / tfin)

    else:
        ct = cos(1.5 * pi * time_of_day / day_length)

        # Return Temp
        return 0.5 * (tmin + tmax - (tmax - tmin) * ct)

#### __Example: calculate the temperature diurnal pattern__

In [ ]:
calculate_temperature_diurnal_pattern(time_of_day=0 ,
                                      day_length=0,
                                      tmin=0.6,
                                      tmax=8.2,
                                      tmin_prev=2.6,
                                      tmax_prev=11.3,
                                      tmin_next=4.7,
                                      )

/tmp/ipykernel_8746/3126062202.py:16: UserWarning: Double check that time_of_day and day_length are in seconds
  warnings.warn("Double check that time_of_day and day_length are in seconds")


4.3999999999999995

In [ ]:
# | export


def calculate_rh_diurnal_pattern(
    temperature: float,  # Unknown parameter definition
    rhmin: float,  # Unknown parameter definition
    rhmax: float,  # Unknown parameter definition
    tmin: float,  # Unknown parameter definition
    tmax: float,  # Unknown parameter definition
) -> float:
    "Calculate diurnal pattern of relative humidity from temperature"

    # calculate rh diurnal pattern ----------------------------------------------
    return rhmax + ((temperature - tmin) / (tmax - tmin)) * (rhmin - rhmax)

In [ ]:
# | export


def rg_watt_ppfd_umol_conversions(
    ppfd: float = None,  # Photosynthetic photon flux density (umol.m-2.s-1)
    rg: float = None,  # Global radiation (W/m2)
    j_to_mol: float = 4.6,  # Conversion factor
    frac_par: float = 0.5,  # Function of solar rdiation that is photosynthetically active radiation (PAR)
    selected_conversion: str = [ # String indicating to what units rg should be converted
        "rg_watts_to_ppfd_umol",
        "ppfd_umol_to_rg_watts",
    ],
) -> float:
    "Convert Global Radiation (rg) in watts to Photosynthetic Photon Flux Density (ppfd) in umol and viceversa"

    # Assert parameters ---------------------------------------------------------

    # Make sure that selected_conversion only has three options
    assert (
        selected_conversion in ["ppfd_umol_to_rg_watts", "rg_watts_to_ppfd_umol"]
    ), f'{selected_conversion} not a valid option for selected_conversion, select "ppfd_umol_to_rg_watts" or "rg_watts_to_ppfd_umol"'

    # Make sure the necessary parameters for a given conversion are provided
    if selected_conversion == "ppfd_umol_to_rg_watts":
        assert isinstance(ppfd, float) | isinstance(
            ppfd, int) | isinstance(ppfd, np.ndarray), "ppfd missing. Parameter must be a float or integer value"

    elif selected_conversion == "rg_watts_to_ppfd_umol":
        assert isinstance(rg, float) | isinstance(
            rg, int) | isinstance(rg, np.ndarray), "rg missing. Parameter must be a float or integer value"

    # Warn in case j_to_mol or frac_par are not provided
    if j_to_mol == 4.6:
        warnings.warn("Using j_to_mol default value of 4.6")

    if frac_par == 0.5:
        warnings.warn("Using frac_par default value of 0.5")

    # Conversions ---------------------------------------------------------------

    # Calculate rg from ppfd (rg)(W/m2)
    if selected_conversion == "ppfd_umol_to_rg_watts":
        print("Conversion of ppfd to rg")
        return ppfd / frac_par / j_to_mol

    # Calculate ppfd (umol.m-2.s-1) from rg
    elif selected_conversion == "rg_watts_to_ppfd_umol":
        print("Conversion of rg to ppfd")
        return rg * frac_par * j_to_mol

    else:
        raise ValueError("Conversion failed")

#### __Example: Calculate RG and PPFD__

In [ ]:
rg_watt_ppfd_umol_conversions(
    rg=7, j_to_mol=4.6, frac_par=0.5, selected_conversion="rg_watts_to_ppfd_umol"
)

Conversion of rg to ppfd


/tmp/ipykernel_11215/1508789894.py:34: UserWarning: Using j_to_mol default value of 4.6
  warnings.warn("Using j_to_mol default value of 4.6")
/tmp/ipykernel_11215/1508789894.py:37: UserWarning: Using frac_par default value of 0.5
  warnings.warn("Using frac_par default value of 0.5")


16.099999999999998

In [ ]:
rg_watt_ppfd_umol_conversions(
    ppfd=9,
    j_to_mol=4.6,
    frac_par=0.5,
    selected_conversion="ppfd_umol_to_rg_watts",
)

Conversion of ppfd to rg


/tmp/ipykernel_11215/1508789894.py:34: UserWarning: Using j_to_mol default value of 4.6
  warnings.warn("Using j_to_mol default value of 4.6")
/tmp/ipykernel_11215/1508789894.py:37: UserWarning: Using frac_par default value of 0.5
  warnings.warn("Using frac_par default value of 0.5")


3.91304347826087

In [ ]:
# | export


def rg_units_conversion(
    rg_watts: float = None,  # instantaneous radiation (watt)
    rg_mj: float = None,  # instantaneous radiation (in Mega Jule?)
    nhours: float = None,  # Unknown parameter definition
    selected_conversion: str = [ # String indicating to what units rg should be converted
        "watts_to_mj",
        "mj_to_watts",
        "mj_to_watts_hour",
    ],
) -> float:
    "Convert instantaneous radiation in watt to dialy cumulative radiation in MJ (MJ.day-1) and viceversa"

    # Assert parameters ---------------------------------------------------------

    # Make sure that selected_conversion only has three options
    assert (
        selected_conversion in ["watts_to_mj", "mj_to_watts", "mj_to_watts_hour"]
    ), f'{selected_conversion} not a valid option for selected_conversion, select "watts_to_mj","mj_to_watts" or "mj_to_watts_hour"'

    # Make sure the necessary parameters for a given conversion are provided
    if selected_conversion == "watts_to_mj":
        assert isinstance(rg_watts, float) | isinstance(
            rg_watts, int) | isinstance(rg_watts, np.ndarray), "rg_watts missing. Parameter must be a float or integer value"

    elif selected_conversion == "mj_to_watts":
        assert isinstance(rg_mj, float) | isinstance(
            rg_mj, int) | isinstance(rg_mj, np.ndarray), "rg_mj missing. Parameter must be a float or integer value"

    elif selected_conversion == "mj_to_watts_hour":
        assert isinstance(rg_mj, float) | isinstance(
            rg_mj, int) | isinstance(rg_mj, np.ndarray) , "rg_mj missing. Parameter must be a float or integer value"

        assert isinstance(nhours, float) | isinstance(
            nhours, int) | isinstance(nhours, np.ndarray) , "nhours missing. Parameter must be a float or integer value"

    # Conversions ---------------------------------------------------------------
    if selected_conversion == "watts_to_mj":
        print("Conversion of rg from watts to Mega Jules per day")

        # Conversion from watts to Mega Jules
        return rg_watts * 0.0864

    elif selected_conversion == "mj_to_watts":
        print("Conversion of rg from Mega Jules per day to Watts")

        # Conversion from Mega Jules to watts
        return rg_mj * (1 / 0.0864)

    elif selected_conversion == "mj_to_watts_hour":
        print("Conversion of rg from Mega Jules to Watts per hour")

        # Conversion from Mega Jules to watts/hour
        return rg_mj * (10**6 / (nhours * 3600))

    else:
        raise ValueError("rg units conversion failed")

#### __Example: RG conversions__

In [ ]:
rg_units_conversion(rg_watts=1.5, selected_conversion="watts_to_mj")

Conversion of rg from watts to Mega Jules per day


0.1296

In [ ]:
rg_units_conversion(rg_mj=1, selected_conversion="mj_to_watts")

Conversion of rg from Mega Jules per day to Watts


11.574074074074073

In [ ]:
rg_units_conversion(rg_mj=np.array([1,2]), nhours=1, selected_conversion="mj_to_watts_hour")

Conversion of rg from Mega Jules to Watts per hour


array([277.77777778, 555.55555556])

In [ ]:
# | export


def declination(
    day_of_year: int,  # julian day (day of the year)
    day_of_spring: int = 80,  # Julian day representing the first day of spring
) -> float:  # Earth declination at day_of_year
    "Calculate declination of sun (radians ? ) for a given julian day (DOY)"

    # Hervé's formula for solar declination

    # Sin(23.5*pi/180), 23.5 = Earth declination

    # Assert parameters ---------------------------------------------------------

    # Day of year
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(day_of_year),
        0,
        err_msg="\nday_of_year must be must be a integer value between 1-366\n",
    )

    np.testing.assert_array_less(
        np.array(day_of_year),
        367,
        err_msg="\nError: day_of_year must be must be a integer value between 1-366\n",
    )

    # date_of_spring
    assert isinstance(
        day_of_spring, int
    ), "day_of_spring must be must be a integer value i.e. 80, 90, 1"

    # Constans ------------------------------------------------------------------

    c1 = 0.398749068925246

    c2 = 2 * 3.1416 / 365

    # date of spring
    day_of_spring_c3 = day_of_spring
    warnings.warn(
        f"date of spring set to {day_of_spring_c3}. This might change for Australia"
    )

    x = c1 * sin((day_of_year - day_of_spring_c3) * c2)

    # Return declination --------------------------------------------------------
    return arctan(x / ((1 - x * x) ** 0.5))

#### __Example: Calculate declination__

In [ ]:
declination(day_of_year=90, day_of_spring=80)

/tmp/ipykernel_11215/2588498006.py:48: UserWarning: date of spring set to 80. This might change for Australia
  warnings.warn(


0.06835636102670284

In [ ]:
declination(day_of_year=np.array([100, 200]), day_of_spring=80)

/tmp/ipykernel_11215/2588498006.py:48: UserWarning: date of spring set to 80. This might change for Australia
  warnings.warn(


array([0.1349969 , 0.35853539])

In [ ]:
# | export


def potential_par(
    time_of_day_in_hours: float,  # Array containing the time of the day (in hours) for which potential par should be calculated
    latitude: float,  # Numeric value specifying the geographic latitude (in decimal degrees) of the location of interest
    day_of_year: int,  # Julian day (day of the year)
) -> np.array:  # Potential Photosynthetic Active Radiation (PAR) for each time_of_day at given latitude and given day_of_year
    "Determine potential for a given place and date /used to determine cloud cover return potential par in W.m2"


    warnings.warn("Make sure time of day is hours in potential_par function")

    # Assert parameters ---------------------------------------------------------

    # time_of_day
    np.testing.assert_array_compare(
        operator.__ge__,
        np.array(time_of_day_in_hours),
        0,
        err_msg="\ntime_of_day must be must equal or greater than 0\n",
    )

    np.testing.assert_array_less(
        np.array(time_of_day_in_hours),
        25,
        err_msg="\nError: time_of_day must be must be equal or lower than 24\n",
    )

    # Latitude
    assert (
        isinstance(latitude, float) | isinstance(latitude, int)
        and 95 >= latitude >= -95
    ), "Provide latitude as coordinates points bewteen -90 and 90 i.e. latitude = 41.40338"

    # Day of year
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(day_of_year),
        0,
        err_msg="\nday_of_year must be must be a integer value between 1-366\n",
    )

    np.testing.assert_array_less(
        np.array(day_of_year),
        367,
        err_msg="\nError: day_of_year must be must be a integer value between 1-366\n",
    )

    # Calculate declination -----------------------------------------------------
    decl = declination(day_of_year=day_of_year, day_of_spring=80)

    # Define constants ----------------------------------------------------------
    diffuse_fraction = 0.1
    solar_constant = 2084
    attenuation_coef = -0.174353387144778

    pn = -cos(latitude * 3.1416 / 180)

    pz = sin(latitude * 3.1416 / 180)

    h_rad = (time_of_day_in_hours - 6) * 3.1416 / 12

    se = cos(h_rad) * cos(decl)
    sn = -pz * sin(h_rad) * cos(decl) - pn * sin(decl)
    sz = -pn * sin(h_rad) * cos(decl) + pz * sin(decl)

    alt = arctan(sz / ((se * se + sn * sn) ** 0.5))
    azi = 3.1416 + arctan(se / sn)

    # Calculate potential par ---------------------------------------------------

    # Step implemented in case azi is a single value i.e. azi = 39
    if isinstance(azi, float):
        # Transform float into array
        azi = np.array([azi])

    azi[sn > 0] = azi[sn > 0] + 3.1416

    pfd = solar_constant * exp(attenuation_coef / sin(alt))

    # Step implemented in case pfd is a single value i.e. pdf = 41
    if isinstance(pfd, float):
        # Transform float into array
        pfd = np.array([pfd])

    pfd[alt < 0] = 0

    dpfd = diffuse_fraction * pfd
    dpfd[alt < 0] = 0

    return dpfd + pfd * sin(alt)

#### __Example: Calculate the potential par__

In [ ]:
potential_par(time_of_day_in_hours=19, latitude=45.1, day_of_year=100)

/tmp/ipykernel_11215/2426213357.py:12: UserWarning: Make sure time of day is hours in potential_par function
  warnings.warn("Make sure time of day is hours in potential_par function")
/tmp/ipykernel_11215/2588498006.py:48: UserWarning: date of spring set to 80. This might change for Australia
  warnings.warn(


array([0.])

In [ ]:
potential_par(time_of_day_in_hours=np.arange(0, 24), latitude=45.1, day_of_year=100)

/tmp/ipykernel_11215/2426213357.py:12: UserWarning: Make sure time of day is hours in potential_par function
  warnings.warn("Make sure time of day is hours in potential_par function")
/tmp/ipykernel_11215/2588498006.py:48: UserWarning: date of spring set to 80. This might change for Australia
  warnings.warn(


array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,   65.37392013,  417.36686646,
        767.72246162, 1069.88738085, 1301.85871798, 1447.6815046 ,
       1497.41611937, 1447.67872923, 1301.85335614, 1069.87979874,
        767.7131865 ,  417.35663351,   65.36575629,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])